# Semantic Kernel 

ในตัวอย่างโค้ดนี้ คุณจะใช้ [Semantic Kernel](https://aka.ms/ai-agents-beginners/semantic-kernel) AI Framework เพื่อสร้างตัวแทนพื้นฐาน

เป้าหมายของตัวอย่างนี้คือการแสดงขั้นตอนที่เราจะใช้ในตัวอย่างโค้ดเพิ่มเติมในภายหลังเมื่อดำเนินการรูปแบบตัวแทนต่างๆ


## นำเข้าแพ็คเกจ Python ที่จำเป็น


In [ ]:
import json
import os

from typing import Annotated

from dotenv import load_dotenv

from IPython.display import display, HTML

from openai import AsyncOpenAI

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.contents import FunctionCallContent, FunctionResultContent, StreamingTextContent
from semantic_kernel.functions import kernel_function

## การสร้าง Client

ในตัวอย่างนี้ เราจะใช้ [GitHub Models](https://aka.ms/ai-agents-beginners/github-models) เพื่อเข้าถึง LLM

`ai_model_id` ถูกกำหนดเป็น `gpt-4o-mini` ลองเปลี่ยนโมเดลเป็นโมเดลอื่นที่มีอยู่ในตลาด GitHub Models เพื่อดูผลลัพธ์ที่แตกต่างกัน

เพื่อให้เราใช้ `Azure Inference SDK` ซึ่งใช้สำหรับ `base_url` ของ GitHub Models เราจะใช้ตัวเชื่อมต่อ `OpenAIChatCompletion` ภายใน Semantic Kernel นอกจากนี้ยังมี [ตัวเชื่อมต่ออื่นๆ](https://learn.microsoft.com/semantic-kernel/concepts/ai-services/chat-completion) ที่สามารถใช้ Semantic Kernel กับผู้ให้บริการโมเดลรายอื่นได้


In [ ]:
import random   

# Define a sample plugin for the sample

class DestinationsPlugin:
    """A List of Random Destinations for a vacation."""

    def __init__(self):
        # List of vacation destinations
        self.destinations = [
            "Barcelona, Spain",
            "Paris, France",
            "Berlin, Germany",
            "Tokyo, Japan",
            "Sydney, Australia",
            "New York, USA",
            "Cairo, Egypt",
            "Cape Town, South Africa",
            "Rio de Janeiro, Brazil",
            "Bali, Indonesia"
        ]
        # Track last destination to avoid repeats
        self.last_destination = None

    @kernel_function(description="Provides a random vacation destination.")
    def get_random_destination(self) -> Annotated[str, "Returns a random vacation destination."]:
        # Get available destinations (excluding last one if possible)
        available_destinations = self.destinations.copy()
        if self.last_destination and len(available_destinations) > 1:
            available_destinations.remove(self.last_destination)

        # Select a random destination
        destination = random.choice(available_destinations)

        # Update the last destination
        self.last_destination = destination

        return destination

In [ ]:
load_dotenv()
client = AsyncOpenAI(
    api_key=os.environ.get("GITHUB_TOKEN"), 
    base_url="https://models.inference.ai.azure.com/",
)

# Create an AI Service that will be used by the `ChatCompletionAgent`
chat_completion_service = OpenAIChatCompletion(
    ai_model_id="gpt-4o-mini",
    async_client=client,
)

## การสร้าง Agent

ด้านล่างนี้เราจะสร้าง Agent ที่ชื่อว่า `TravelAgent`

สำหรับตัวอย่างนี้ เราใช้คำแนะนำที่เรียบง่ายมาก คุณสามารถเปลี่ยนคำแนะนำเหล่านี้เพื่อดูว่า Agent ตอบสนองแตกต่างกันอย่างไร


In [ ]:
AGENT_INSTRUCTIONS = """You are a helpful AI Agent that can help plan vacations for customers.

Important: When users specify a destination, always plan for that location. Only suggest random destinations when the user hasn't specified a preference.

When the conversation begins, introduce yourself with this message:
"Hello! I'm your TravelAgent assistant. I can help plan vacations and suggest interesting destinations for you. Here are some things you can ask me:
1. Plan a day trip to a specific location
2. Suggest a random vacation destination
3. Find destinations with specific features (beaches, mountains, historical sites, etc.)
4. Plan an alternative trip if you don't like my first suggestion

What kind of trip would you like me to help you plan today?"

Always prioritize user preferences. If they mention a specific destination like "Bali" or "Paris," focus your planning on that location rather than suggesting alternatives.
"""

agent = ChatCompletionAgent(
    service=chat_completion_service, 
    plugins=[DestinationsPlugin()],
    name="TravelAgent",
    instructions=AGENT_INSTRUCTIONS,
)

## การรันเอเจนต์

ตอนนี้เราสามารถรันเอเจนต์ได้โดยการกำหนด `ChatHistory` และเพิ่ม `system_message` เข้าไปในนั้น เราจะใช้ `AGENT_INSTRUCTIONS` ที่เราได้กำหนดไว้ก่อนหน้านี้

หลังจากที่กำหนดสิ่งเหล่านี้แล้ว เราจะสร้าง `user_inputs` ซึ่งจะเป็นสิ่งที่ผู้ใช้ส่งไปยังเอเจนต์ ในกรณีนี้ เราได้ตั้งค่าข้อความนี้เป็น `Plan me a sunny vacation`

คุณสามารถเปลี่ยนข้อความนี้ได้ตามต้องการเพื่อดูว่าเอเจนต์จะตอบสนองแตกต่างกันอย่างไร


In [ ]:
user_inputs = [
    "Plan me a day trip.",
    "I don't like that destination. Plan me another vacation.",
]

async def main():
    thread: ChatHistoryAgentThread | None = None

    for user_input in user_inputs:
        html_output = (
            f"<div style='margin-bottom:10px'>"
            f"<div style='font-weight:bold'>User:</div>"
            f"<div style='margin-left:20px'>{user_input}</div></div>"
        )

        agent_name = None
        full_response: list[str] = []
        function_calls: list[str] = []

        # Buffer to reconstruct streaming function call
        current_function_name = None
        argument_buffer = ""

        async for response in agent.invoke_stream(
            messages=user_input,
            thread=thread,
        ):
            thread = response.thread
            agent_name = response.name
            content_items = list(response.items)

            for item in content_items:
                if isinstance(item, FunctionCallContent):
                    if item.function_name:
                        current_function_name = item.function_name

                    # Accumulate arguments (streamed in chunks)
                    if isinstance(item.arguments, str):
                        argument_buffer += item.arguments
                elif isinstance(item, FunctionResultContent):
                    # Finalize any pending function call before showing result
                    if current_function_name:
                        formatted_args = argument_buffer.strip()
                        try:
                            parsed_args = json.loads(formatted_args)
                            formatted_args = json.dumps(parsed_args)
                        except Exception:
                            pass  # leave as raw string

                        function_calls.append(f"Calling function: {current_function_name}({formatted_args})")
                        current_function_name = None
                        argument_buffer = ""

                    function_calls.append(f"\nFunction Result:\n\n{item.result}")
                elif isinstance(item, StreamingTextContent) and item.text:
                    full_response.append(item.text)

        if function_calls:
            html_output += (
                "<div style='margin-bottom:10px'>"
                "<details>"
                "<summary style='cursor:pointer; font-weight:bold; color:#0066cc;'>Function Calls (click to expand)</summary>"
                "<div style='margin:10px; padding:10px; background-color:#f8f8f8; "
                "border:1px solid #ddd; border-radius:4px; white-space:pre-wrap; font-size:14px; color:#333;'>"
                f"{chr(10).join(function_calls)}"
                "</div></details></div>"
            )

        html_output += (
            "<div style='margin-bottom:20px'>"
            f"<div style='font-weight:bold'>{agent_name or 'Assistant'}:</div>"
            f"<div style='margin-left:20px; white-space:pre-wrap'>{''.join(full_response)}</div></div><hr>"
        )

        display(HTML(html_output))

await main()


---

**ข้อจำกัดความรับผิดชอบ**:  
เอกสารนี้ได้รับการแปลโดยใช้บริการแปลภาษา AI [Co-op Translator](https://github.com/Azure/co-op-translator) แม้ว่าเราจะพยายามให้การแปลมีความถูกต้องมากที่สุด แต่โปรดทราบว่าการแปลอัตโนมัติอาจมีข้อผิดพลาดหรือความไม่ถูกต้อง เอกสารต้นฉบับในภาษาดั้งเดิมควรถือเป็นแหล่งข้อมูลที่เชื่อถือได้ สำหรับข้อมูลที่สำคัญ ขอแนะนำให้ใช้บริการแปลภาษามืออาชีพ เราไม่รับผิดชอบต่อความเข้าใจผิดหรือการตีความผิดที่เกิดจากการใช้การแปลนี้
